In [11]:
import keras
import os
import cv2
import numpy as np

TYPE = "5_resnet50"

MODELNAME = 'data\\model_out\\build_' + TYPE + '.h5'
DATASET_IMGS = 'data\\dataset_test'
HTML_FILE = 'data\\model_out\\output_build_' + TYPE + '.html'
IMG_SHAPE = (180, 320)
CL = ['GRA', 'HIS', 'IN', 'STUD', 'MIX', 'OUT_C', 'OUT_H', 'OTHER', 'SPE']

In [12]:
model = keras.models.load_model(MODELNAME)
with open(HTML_FILE, 'w') as file:
    classes = range(0, len(CL))
    
    file.write('<html>\n\t<body>\n')
    file.write('\t\t<table border="2">\n')
    file.write('\t\t\t<tr>\n')
    file.write('\t\t\t\t<td>Image</td>\n')
    [file.write('\t\t\t\t<td style="min-width:100px">' + CL[i] + '</td>\n') for i in classes]
    file.write('\t\t\t\t<td>Classification</td>\n')
    file.write('\t\t\t</tr>\n')
    
    for img_name in os.listdir(DATASET_IMGS):
        img = cv2.imread(DATASET_IMGS + '\\' + img_name)
        img = cv2.resize(img, (IMG_SHAPE[1], IMG_SHAPE[0]), interpolation=cv2.INTER_CUBIC)
        img = np.array([img])
        img = img.astype("float32")
        img[0, ...] = np.divide(img[0, ...], 255.0)
        
        network_say = model.predict(img)[0]
        
        file.write('\t\t\t<tr>\n')
        file.write('\t\t\t\t<td><img src="..\\dataset_test\\' + img_name + '" width="' + str(IMG_SHAPE[1] * 2) + '" height="' + str(IMG_SHAPE[0] * 2) + '"></td>\n')
        [file.write('\t\t\t\t<td>' + ('<b>' if network_say[i] == max(network_say) else '') + 
                    format("%.6f" % (round(network_say[i] * 1_000_000) / 1_000_000)) +
                    ('</b>' if network_say[i] == max(network_say) else '') + '</td>\n') for i in classes]
        file.write('\t\t\t\t<td>(' + CL[np.argmax(network_say)] + ')')
        file.write('\t\t\t</tr>\n')
        
    file.write('\t\t</table>\n')
    file.write('\t</body>\n</html>')
